In [1]:
# pandasとtrain_test_splitを呼び出し
import pandas as pd
from sklearn.model_selection import train_test_split
# データ読み込み
X = pd.read_csv('train.csv', index_col='Id')
X_test = pd.read_csv( 'test.csv', index_col='Id')

# 'SalePrice'のない行を削除
# inplace指定で元のdata frameを置き換える
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
# 目的変数と説明変数に分割
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

## 欠損値のあるデータを削除
# 欠損値のある列名を抽出
cols_with_missing = [col for col in X.columns
                     if X[col].isnull().any()]
# 訓練用、テスト用からそれぞれ欠損値のある列を落とす
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)
# 訓練データと検証データに分ける
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
# データを確認
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


In [2]:
# ランダムフォレストとMAEを呼び出し
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
# モデルのスコアリングを行う関数を作成
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Step 1:Drop columns with categorical data

In [3]:
#カラムごとにデータ型を表示
X_train.dtypes

MSSubClass        int64
MSZoning         object
LotArea           int64
Street           object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
OverallQual       int64
OverallCond       int64
YearBuilt         int64
YearRemodAdd      int64
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
ExterQual        object
ExterCond        object
Foundation       object
BsmtFinSF1        int64
BsmtFinSF2        int64
BsmtUnfSF         int64
TotalBsmtSF       int64
Heating          object
HeatingQC        object
CentralAir       object
1stFlrSF          int64
2ndFlrSF          int64
LowQualFinSF      int64
GrLivArea         int64
BsmtFullBath      int64
BsmtHalfBath      int64
FullBath          int64
HalfBath          int64
BedroomAbvGr      int64
KitchenAbvGr    

In [4]:
X_train.dtypes == 'object'

MSSubClass       False
MSZoning          True
LotArea          False
Street            True
LotShape          True
LandContour       True
Utilities         True
LotConfig         True
LandSlope         True
Neighborhood      True
Condition1        True
Condition2        True
BldgType          True
HouseStyle        True
OverallQual      False
OverallCond      False
YearBuilt        False
YearRemodAdd     False
RoofStyle         True
RoofMatl          True
Exterior1st       True
Exterior2nd       True
ExterQual         True
ExterCond         True
Foundation        True
BsmtFinSF1       False
BsmtFinSF2       False
BsmtUnfSF        False
TotalBsmtSF      False
Heating           True
HeatingQC         True
CentralAir        True
1stFlrSF         False
2ndFlrSF         False
LowQualFinSF     False
GrLivArea        False
BsmtFullBath     False
BsmtHalfBath     False
FullBath         False
HalfBath         False
BedroomAbvGr     False
KitchenAbvGr     False
KitchenQual       True
TotRmsAbvGr

In [5]:
# カテゴリカル変数のリストを取得
s = (X_train.dtypes=='object')
s[s]

MSZoning         True
Street           True
LotShape         True
LandContour      True
Utilities        True
LotConfig        True
LandSlope        True
Neighborhood     True
Condition1       True
Condition2       True
BldgType         True
HouseStyle       True
RoofStyle        True
RoofMatl         True
Exterior1st      True
Exterior2nd      True
ExterQual        True
ExterCond        True
Foundation       True
Heating          True
HeatingQC        True
CentralAir       True
KitchenQual      True
Functional       True
PavedDrive       True
SaleType         True
SaleCondition    True
dtype: bool

In [6]:
s[s].index

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [7]:
object_cols = list(s[s].index)

In [8]:
# カテゴリカル変数を削除
drop_X_train = X_train.drop(object_cols, axis=1)
drop_X_valid = X_valid.drop(object_cols, axis=1)
# drop_X_train = X_train.select_dtypes(exclude=['object']) でも同じ結果

# スコアを表示
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


In [9]:
# Condition2の列の値を確認 uniqueを使うと配列型(np.array)で帰ってくる
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())
# setでもよい。集合型(sets)になる。
print(set(X_train['Condition2']))

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']
{'PosN', 'PosA', 'Norm', 'RRAe', 'Artery', 'Feedr'}


# Step 2: Ordinal encoding
## Part A
学習データの列に序数エンコーダを当てはめると、学習データに現れるユニークな値それぞれに対応する整数値のラベルが作成されます。検証データに学習データにもない値が含まれている場合、これらの値には整数が割り当てられないので、エンコーダはエラーを投げます。検証データの 'Condition2' カラムには 'RRAn' と 'RRNn' という値がありますが、これらは学習データには現れないので、 scikit-learn で序数エンコーダを使おうとするとエラーになります。

In [10]:
# カテゴリカル変数の列を抽出
object_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
# X_validの要素がX_trainの要素に含まれるものを抽出(issubset または <= で判別)
good_label_cols = [col for col in object_cols
                   if set(X_valid[col]).issubset(set(X_train[col]))]
# X_validの要素でX_trainにないものを抽出
bad_label_cols = list(set(object_cols) - set(good_label_cols))
# それぞれの中身を表示
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be ordinal encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Condition2', 'Functional', 'RoofMatl']


## Part B

In [11]:
# Ordinal encoderをインポート
from sklearn.preprocessing import OrdinalEncoder
# bad_labelになったものを落とす
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)
# ordinal encoderを定義
ordinal_encoder = OrdinalEncoder()
# ordinal encoderで
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])

print("MAE from Approach 2 (Ordinal Encoding):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
17098.01649543379


これまで、あなたはカテゴリ変数を扱うために2つの異なるアプローチを試してきました。そして、カテゴリデータをエンコードすることで、データセットから列を削除するよりも良い結果が得られることがわかりました。

近々、ワンホットエンコーディングを試してみることにしましょう。その前に、もう一つ説明しなければならないことがあります。次のコードセルを変更せずに実行することから始めてください。

In [12]:
## カテゴリカル変数の列それぞれのデータの種類をカウント
# map関数と無名関数でリスト内に関数を適応 map(関数, イテラブル)
# .nuniqueでユニークの個数を取得
# mapはリストに変換しないと参照できない
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
# zipで2つのリストから辞書を作る
d = dict(zip(object_cols, object_nunique))

# 辞書の並び替え
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

# Step 3: Investigating cardinality
## Part A
上の出力は、カテゴリデータを持つ各列について、その列のユニークな値の数を示しています。例えば、学習データの'Street'列には2つのユニークな値があります。Grvl'と'Pave'であり、それぞれ砂利道と舗装路に相当する。
我々は、カテゴリ変数のユニークなエントリの数を、そのカテゴリ変数の基数（cardinality）と呼ぶことにする。たとえば，'Street' 変数は，基数 2 を持つ．
## Part B
多くの行を持つ大規模なデータセットでは、ワンホットエンコーディングを行うと、データセットのサイズが大きく膨らんでしまうことがあります。このため、通常、比較的カーディナリティの低い列のみをワンホットエンコードすることにしている。そして、カーディナリティの高い列はデータセットから削除するか、序数エンコードを使用することができる。
次に、ワンホットエンコーディングの実験を行います。しかし、データセットのすべてのカテゴリ変数をエンコードするのではなく、カーディナリティが10未満の列に対してのみ、ワンホットエンコーディングを作成します。
以下のコードを変更せずに実行して、low_cardinality_colsをワンホットエンコーディングされる列を含むPythonのリストに設定します。同様に、high_cardinality_colsには、データセットから削除されるカテゴリカルカラムのリストが格納されます。

In [13]:
# object_colsの中でユニーク個数の10未満のものを出力
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# 10以上のものも出力
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Neighborhood', 'Exterior2nd', 'Exterior1st']


# Step 4: One-hot encoding

次のコードセルを使って、X_train と X_valid のデータをワンホットエンコードする。前処理したDataFrameをそれぞれOH_X_trainとOH_X_validにセットします。
データセットのカテゴリカラムの全リストは、Pythonのリストobject_colsに記載されています。
low_cardinality_colsにあるカテゴリカルカラムだけをワンホットエンコードする必要があります。それ以外のカテゴリカルカラムはデータセットから削除する必要があります。

In [18]:
from operator import concat
from sklearn.preprocessing import OneHotEncoder

#  one hot encoderを定義
# sparseではなくnumpy matrixにする
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
# リスト型で返ってくるのでdata frameにしておく
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))
# indexが落ちているので再入力
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# カテゴリカル変数を一旦全て削除
notOH_X_train = X_train.drop(object_cols, axis=1)
notOH_X_valid = X_valid.drop(object_cols, axis=1)
# one hotでエンコードした列のみ再度加える
OH_X_train = pd.concat([notOH_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([notOH_X_valid, OH_cols_valid], axis=1)

print(OH_X_train.head())
print(OH_X_valid.head())

     MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  \
Id                                                                            
619          20    11694            9            5       2007          2007   
871          20     6600            5            5       1962          1962   
93           30    13360            5            7       1921          2006   
818          20    13265            8            5       2002          2002   
303          20    13704            7            5       2001          2002   

     BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  ...  112  113  114  115  \
Id                                                   ...                       
619          48           0       1774         1822  ...  0.0  1.0  0.0  0.0   
871           0           0        894          894  ...  0.0  0.0  0.0  1.0   
93          713           0        163          876  ...  0.0  0.0  0.0  1.0   
818        1218           0        350        

In [20]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):


/Users/htmbp2022/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


17525.345719178084


/Users/htmbp2022/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


# まとめ